In [2]:


import numpy as np
import pandas as pd



from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error



# Download the two parquets

In [3]:

# URL of the Parquet file
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"

# Download the file locally
response = requests.get(url)
file_path_jan = "data/yellow_tripdata_2023-01.parquet"

# Save the downloaded content to a local file
with open(file_path_jan, 'wb') as file:
    file.write(response.content)

# Read the Parquet file into a pandas DataFrame
df_jan = pd.read_parquet(file_path_jan)




import pandas as pd
import requests

# URL of the Parquet file
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet"

# Download the file locally
response = requests.get(url)
file_path_feb = "data/yellow_tripdata_2023-02.parquet"

# Save the downloaded content to a local file
with open(file_path_feb, 'wb') as file:
    file.write(response.content)

# Read the Parquet file into a pandas DataFrame
df_feb = pd.read_parquet(file_path_feb)




In [3]:
file_path_feb = "data/yellow_tripdata_2023-02.parquet"
file_path_jan = "data/yellow_tripdata_2023-01.parquet"

df_jan = pd.read_parquet(file_path_jan)
df_feb = pd.read_parquet(file_path_feb)

## How many cols are there?

In [4]:
print("Number of columns in df_jan:", df_jan.shape[1])

Number of columns in df_jan: 19


In [5]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [6]:

df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)

std_duration = df_jan['duration'].std()
print("std:", std_duration)
print("mean: ",df_jan['duration'].mean())    


std: 42.59435124195458
mean:  15.668995167330452


In [7]:

df_feb['duration'] = df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime
df_feb.duration = df_feb.duration.apply(lambda td: td.total_seconds() / 60)

std_duration = df_feb['duration'].std()
print("std:", std_duration)
print("mean: ",df_feb['duration'].mean())    

std: 42.84210176105113
mean:  16.01590961997241


In [8]:
df_jan_filtered = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]

# Calculate the number of rows in df_jan_filtered
num_rows_filtered = len(df_jan_filtered)

# Calculate the total number of rows in df_jan
num_rows_total = len(df_jan)

# Calculate the percentage of rows left after the filter
percentage_left = (num_rows_filtered / num_rows_total) * 100

# Print the percentage
print(f"Percentage of rows left after the filter: {percentage_left:.2f}%")

Percentage of rows left after the filter: 98.12%


In [9]:
df_feb_filtered = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]
# Calculate the number of rows in df_feb_filtered
num_rows_filtered = len(df_feb_filtered)

# Calculate the total number of rows in df_feb
num_rows_total = len(df_feb)

# Calculate the percentage of rows left after the filter
percentage_left = (num_rows_filtered / num_rows_total) * 100

# Print the percentage
print(f"Percentage of rows left after the filter: {percentage_left:.2f}%")

Percentage of rows left after the filter: 98.01%


In [10]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_jan[categorical] = df_feb[categorical].astype(str)

In [11]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_feb[categorical] = df_feb[categorical].astype(str)

In [17]:
df_features = pd.concat([df_jan, df_feb])
df_features

df_features.dropna(subset=['DOLocationID', 'PULocationID'], inplace=True)

df_features

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration,Airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,142,163,2,...,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333,NaN
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,71,71,1,...,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667,NaN
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,71,71,1,...,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000,NaN
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,132,26,1,...,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667,NaN
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,161,145,1,...,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913950,2,2023-02-28 23:46:00,2023-03-01 00:05:00,NaN,4.65,NaN,None,249,140,0,...,0.00,0.5,4.84,0.0,1.0,29.06,NaN,NaN,19.000000,NaN
2913951,2,2023-02-28 23:26:02,2023-02-28 23:37:10,NaN,2.47,NaN,None,186,79,0,...,0.00,0.5,2.65,0.0,1.0,20.31,NaN,NaN,11.133333,NaN
2913952,2,2023-02-28 23:24:00,2023-02-28 23:38:00,NaN,3.49,NaN,None,158,143,0,...,0.00,0.5,0.00,0.0,1.0,21.64,NaN,NaN,14.000000,NaN
2913953,2,2023-02-28 23:03:00,2023-02-28 23:10:00,NaN,2.13,NaN,None,79,162,0,...,0.00,0.5,2.63,0.0,1.0,20.19,NaN,NaN,7.000000,NaN


In [18]:
train_dicts = df_features[categorical + numerical].to_dict(orient='records')
print(train_dicts[:5])

[{'PULocationID': '142', 'DOLocationID': '163', 'trip_distance': 0.97}, {'PULocationID': '71', 'DOLocationID': '71', 'trip_distance': 1.1}, {'PULocationID': '71', 'DOLocationID': '71', 'trip_distance': 2.51}, {'PULocationID': '132', 'DOLocationID': '26', 'trip_distance': 1.9}, {'PULocationID': '161', 'DOLocationID': '145', 'trip_distance': 1.43}]


In [19]:

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df_features[target].values

feature_names = dv.feature_names_
print(feature_names)

X_train

['DOLocationID=1', 'DOLocationID=10', 'DOLocationID=100', 'DOLocationID=101', 'DOLocationID=102', 'DOLocationID=105', 'DOLocationID=106', 'DOLocationID=107', 'DOLocationID=108', 'DOLocationID=109', 'DOLocationID=11', 'DOLocationID=111', 'DOLocationID=112', 'DOLocationID=113', 'DOLocationID=114', 'DOLocationID=115', 'DOLocationID=116', 'DOLocationID=117', 'DOLocationID=118', 'DOLocationID=119', 'DOLocationID=12', 'DOLocationID=120', 'DOLocationID=121', 'DOLocationID=122', 'DOLocationID=123', 'DOLocationID=124', 'DOLocationID=125', 'DOLocationID=126', 'DOLocationID=127', 'DOLocationID=128', 'DOLocationID=129', 'DOLocationID=13', 'DOLocationID=130', 'DOLocationID=131', 'DOLocationID=132', 'DOLocationID=133', 'DOLocationID=134', 'DOLocationID=135', 'DOLocationID=136', 'DOLocationID=137', 'DOLocationID=138', 'DOLocationID=139', 'DOLocationID=14', 'DOLocationID=140', 'DOLocationID=141', 'DOLocationID=142', 'DOLocationID=143', 'DOLocationID=144', 'DOLocationID=145', 'DOLocationID=146', 'DOLoc

<5827910x520 sparse matrix of type '<class 'numpy.float64'>'
	with 17483730 stored elements in Compressed Sparse Row format>

In [21]:

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mse=mean_squared_error(y_train, y_pred, squared=False)
rmse = np.sqrt(mse)
rmse

/Users/santiago/Desktop/Ab-Inbev/mlops-zoomcamp/mlops-zoomcamp-3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


6.5484495312512365

In [25]:
val_dicts = df_feb[categorical + numerical].to_dict(orient='records')
print(val_dicts[:5])

dv = DictVectorizer()
X_val = dv.fit_transform(val_dicts)

target = 'duration'
y_val = df_feb[target].values

feature_names = dv.feature_names_
print(feature_names)

X_val

y_pred = lr.predict(X_val)

mse=mean_squared_error(y_val, y_pred, squared=False)
rmse = np.sqrt(mse)


['DOLocationID=1', 'DOLocationID=10', 'DOLocationID=100', 'DOLocationID=101', 'DOLocationID=102', 'DOLocationID=105', 'DOLocationID=106', 'DOLocationID=107', 'DOLocationID=108', 'DOLocationID=109', 'DOLocationID=11', 'DOLocationID=111', 'DOLocationID=112', 'DOLocationID=113', 'DOLocationID=114', 'DOLocationID=115', 'DOLocationID=116', 'DOLocationID=117', 'DOLocationID=118', 'DOLocationID=119', 'DOLocationID=12', 'DOLocationID=120', 'DOLocationID=121', 'DOLocationID=122', 'DOLocationID=123', 'DOLocationID=124', 'DOLocationID=125', 'DOLocationID=126', 'DOLocationID=127', 'DOLocationID=128', 'DOLocationID=129', 'DOLocationID=13', 'DOLocationID=130', 'DOLocationID=131', 'DOLocationID=132', 'DOLocationID=133', 'DOLocationID=134', 'DOLocationID=135', 'DOLocationID=136', 'DOLocationID=137', 'DOLocationID=138', 'DOLocationID=139', 'DOLocationID=14', 'DOLocationID=140', 'DOLocationID=141', 'DOLocationID=142', 'DOLocationID=143', 'DOLocationID=144', 'DOLocationID=145', 'DOLocationID=146', 'DOLoc

/Users/santiago/Desktop/Ab-Inbev/mlops-zoomcamp/mlops-zoomcamp-3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


6.511311938526858